In [1]:
# 基本配置
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
from langchain_community.embeddings import DashScopeEmbeddings

load_dotenv(override=True)

qw_llm_openai = ChatOpenAI(
    openai_api_base=os.getenv('DASHSCOPE_API_BASE'),
    openai_api_key=os.getenv('DASHSCOPE_API_KEY'),
    model_name="qwen-plus",
    temperature=0,
    streaming=True,
)

qw_embedding = DashScopeEmbeddings(
    model="text-embedding-v2", dashscope_api_key=os.getenv('DASHSCOPE_API_KEY')
)

In [2]:
from langchain_community.document_loaders import PyMuPDFLoader

file_path = '../../../file/qiyu-yichujida.pdf'
docs = PyMuPDFLoader(file_path).load()

In [3]:
docs

[Document(page_content='一触即达 配置操作指南 \n一、 适用场景 \n一触即达主要解决在线机器人面对客户咨询时根据实际场景给予个性化的回答，\n基于多\n个逻辑节点提取客户回答中的关键信息，无需跳转到其他页面，\n即可实现售前客户留资、\n售\n后解决问题、提供业务支持等目的，包括但不限于以下场景： \n1、售前客户留资：客户咨询业务时根据自身特点提供必要信息，在线机器人通过一触\n即达的预先配置提取有效信息，\n为客户提供符合其需求的业务及服务人员，\n同时留存客户联\n系方式，完成售前客户留资，如教育场景课程咨询。 \n2、售后解决问题：客户在产品及服务的使用过程中出现问题咨询时，在线机器人根据\n客户反馈的问题类型进行进一步分类，从而为客户提供其需要的解决方案及寻求帮助的途\n径，同时留存记录客户反馈的问题，如电商场景物流、订单等内容查询。 \n3、提供业务支持：客户在业务流程中存在疑问咨询时，在线机器人根据客户在咨询时\n提供的信息引导其完成业务操作、\n提供相关的业务说明或提供业务支持帮助，\n如政府公积金、\n保险等内容查询。 \n \n二、 配置说明（以留资场景为例） \n留资场景为一触即达功能最常见适用场景，\n因此配置说明选用驾校驾考留资场景流程作\n为配置举例。驾考留资场景涉及城市、驾考类型、手机号、微信号等多个实体，客户仅需提\n前配置在线机器人问答流程，\n根据逻辑问答判断即可实现提取、\n收集、存储客户留资关键信\n息，为后续人工跟进客户情况提供有效帮助，并高效存储客户咨询信息。 \n1、 新增实体和变量 \n新增实体：\n【一触即达】-【实体管理】-【新增实体】- 根据需要选择相关实体 \n新增变量：\n【一触即达】-【流程管理】-【变量管理】-【新增变量】 \n根据需要启用【关联实体】\n，若涉及新实体，关联前需要事先新增实体。 \n注：常见实体如城市、手机等，已由系统默认配置完毕，无需再次新增实体操作。 \n \n', metadata={'source': '../../../file/qiyu-yichujida.pdf', 'file_path': '../../../file/qiyu-yichujida.pdf', 'page': 0, 'total_pages': 11, 'format': 'P

In [4]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0, add_start_index=True)
splits = text_splitter.split_documents(docs)

In [5]:
len(splits)

11

In [6]:
from langchain.vectorstores import Chroma

persist_directory = '/Users/pangmengting/Documents/workspace/python-learning/data/chroma_vector_db'
collection_name = 'chat-pdf-index'
# 创建
vectorstore = Chroma.from_documents(
    splits,
    qw_embedding,
    collection_name=collection_name,
    persist_directory=persist_directory
)

In [7]:
retriever = vectorstore.as_retriever()
retriever.search_kwargs['k'] = 3

In [8]:
from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain

qa = ConversationalRetrievalChain.from_llm(qw_llm_openai, retriever=retriever)

In [10]:
chat_history = []


def ask(question, chat_history):
    response = qa.invoke({"question": question, "chat_history": chat_history})
    chat_history.append((question, response['answer']))
    print(f"Question:\n {question}\n")
    print(f"Answer:\n {response['answer']}\n")

In [11]:
query = '什么是一触即达'

In [12]:
ask(query, chat_history)

Question:
 什么是一触即达

Answer:
 一触即达是一种配置功能，主要用于在线机器人与客户的交互中，根据不同的场景提供个性化回答。它通过逻辑节点提取客户回答的关键信息，无需跳转页面，就能实现售前客户留资、售后问题解决和业务支持等多种目标。例如，在售前，它可以帮助收集客户信息并推荐适合的业务；售后时，可以根据客户问题分类提供解决方案；在业务流程中，可以指导客户完成操作或提供业务查询支持。配置一触即达涉及到实体和变量的新增、意图管理和对话节点设计等步骤。


In [13]:
query = '总结一下这个文档'
ask(query, chat_history)

Question:
 总结一下这个文档

Answer:
 这个文档主要介绍了如何新建流程以及流程中的两个关键节点：查询节点和收集节点。查询节点主要用于通过机器人调用业务系统进行查询，并展示查询结果或作为子节点的判断条件，涉及设置项如请求地址、请求方式等。收集节点则用于收集客户信息，可以对接到工单中心、客户中心或第三方系统，并提供了引导语、支持附件和表单字段等设置。


In [14]:
chat_history

[('什么是一触即达',
  '一触即达是一种配置功能，主要用于在线机器人与客户的交互中，根据不同的场景提供个性化回答。它通过逻辑节点提取客户回答的关键信息，无需跳转页面，就能实现售前客户留资、售后问题解决和业务支持等多种目标。例如，在售前，它可以帮助收集客户信息并推荐适合的业务；售后时，可以根据客户问题分类提供解决方案；在业务流程中，可以指导客户完成操作或提供业务查询支持。配置一触即达涉及到实体和变量的新增、意图管理和对话节点设计等步骤。'),
 ('总结一下这个文档',
  '这个文档主要介绍了如何新建流程以及流程中的两个关键节点：查询节点和收集节点。查询节点主要用于通过机器人调用业务系统进行查询，并展示查询结果或作为子节点的判断条件，涉及设置项如请求地址、请求方式等。收集节点则用于收集客户信息，可以对接到工单中心、客户中心或第三方系统，并提供了引导语、支持附件和表单字段等设置。')]

In [15]:
query = '判断节点是什么？如何设置？'
ask(query, chat_history)

Question:
 判断节点是什么？如何设置？

Answer:
 判断节点在流程中用于通过提取信息的条件判断串联各节点，它可以根据提取的变量来实现机器人的自动回复。设置时，你可以对一个实体值或一个变量值进行判断。以下是设置的关键点：

- 需要修改的设置项可能包括判断条件，这可以是对某个特定实体或变量的检查。
- 你可以根据业务逻辑设置条件，例如，如果用户提供的信息满足特定条件，机器人将执行相应的回复或操作。

由于没有具体的设置界面或详细步骤，我无法提供详细的设置过程。通常，你可能需要在后台编辑器中选择“添加判断节点”，然后配置判断条件、结果分支（当条件为真和为假时的回复或动作）以及可能涉及的变量。如果你有更具体的问题或者需要了解某个平台的详细步骤，可以告诉我，我会尽力帮助解答。


==================================================================================================================

In [24]:
# 新方式
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.history_aware_retriever import create_history_aware_retriever
# from langchain.chains import (create_history_aware_retriever, create_retrieval_chain, )
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Contextualize question
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, just "
    "reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    qw_llm_openai, retriever, contextualize_q_prompt
)

# Answer question
qa_system_prompt = (
    "You are an assistant for question-answering tasks. Use "
    "the following pieces of retrieved context to answer the "
    "question. If you don't know the answer, just say that you "
    "don't know. Use three sentences maximum and keep the answer "
    "concise."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
# Below we use create_stuff_documents_chain to feed all retrieved context
# into the LLM. Note that we can also use StuffDocumentsChain and other
# instances of BaseCombineDocumentsChain.
question_answer_chain = create_stuff_documents_chain(qw_llm_openai, qa_prompt)
rag_chain = create_retrieval_chain(
    history_aware_retriever, question_answer_chain
)

In [ ]:
# Usage:
chat_history = []  # Collect chat history here (a sequence of messages)
rag_chain.invoke({"input": query, "chat_history": chat_history})

In [25]:
chat_history = []


def ask_new(question, chat_history):
    response = rag_chain.invoke({"input": question, "chat_history": chat_history})
    chat_history.append((question, response['answer']))
    print(f"Question:\n {question}\n")
    print(f"Answer:\n {response['answer']}\n")

In [18]:
query = '总结一下这个文档'
ask_new(query, chat_history)

Question:
 总结一下这个文档

Answer:
 这段文档介绍了如何创建新流程和理解不同类型的节点。"新建流程"涉及选择模板或创建空白模板，并输入流程名称。"查询节点"用于机器人调用业务系统进行查询，基于用户信息，返回结果给用户或作为其他节点的判断条件，关键设置包括请求地址、请求方式和输入变量。"收集节点"则用来收集客户信息，可以对接到工单、客户中心或第三方系统，支持设置引导语、是否允许附件上传及自定义表单字段。


In [26]:
query = '回复节点是什么呢'
ask_new(query, chat_history)

Question:
 回复节点是什么呢

Answer:
 回复节点是流程中的一个环节，它用来在特定节点位置提供定制化的回复给用户。你可以选择系统应答样式、系统反问样式或自定义样式来设计回复。此外，可以通过添加富文本、按钮选项来构建回复内容，并能设置点击按钮后跳转到下一节点或外部链接。还可以提取变量，以便在用户选择后用于调用业务接口。
